In [1]:
pip install langchain openai huggingface-hub pypdf chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

os.environ['OPENAI_API_KEY'] = 'YOUR_OPEN_API_KEY'
os.environ['HUGGINGFACE_API_KEY'] = 'HUGGINGFACE_API_KEY'


In [3]:
from langchain.agents import tool

@tool
def symptom_checker(symptom: str) -> str:
    """Check common causes for a given symptom."""
    common_causes = {
        "fever": "Common causes include infections (bacterial or viral), heat exhaustion, and inflammatory conditions.",
        "headache": "Common causes include tension, stress, dehydration, and sinusitis.",
        "cough": "Common causes include colds, flu, bronchitis, and asthma."
    }
    return common_causes.get(symptom.lower(), "Symptom not found in the database. Please consult a healthcare professional.")


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool

# Instantiate the LLM and Tools
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
tools = [Tool(name="SymptomChecker", func=symptom_checker, description="Check common causes for symptoms.")]

# Define Prompts
prompt1 = ChatPromptTemplate.from_template("Check common causes for the symptom: {symptom}")
prompt2 = ChatPromptTemplate.from_template("Summarize the advice in a friendly tone.")

# Create Chains
chain1 = LLMChain(llm=llm, prompt=prompt1)
chain2 = LLMChain(llm=llm, prompt=prompt2)

# Invoking Chains and Printing Results
response1 = chain1.invoke({"symptom": "fever"})
response2 = chain2.invoke({"symptom": response1})

print("Symptom Check:", response1)
print("Summary:", response2)


Symptom Check: {'symptom': 'fever', 'text': "1. Infection: The most common cause of a fever is an infection, such as a bacterial or viral infection. Common examples include the flu, pneumonia, urinary tract infections, and strep throat.\n\n2. Inflammatory conditions: Autoimmune diseases, such as lupus or rheumatoid arthritis, can cause fevers as the body's immune system attacks its own tissues.\n\n3. Medications: Some medications, such as antibiotics, can cause a fever as a side effect.\n\n4. Heat exhaustion or heat stroke: Being exposed to high temperatures for an extended period of time can cause a fever as the body tries to cool itself down.\n\n5. Cancer: Some types of cancer can cause fevers due to the body's immune response to the cancer cells.\n\n6. Immunizations: Some vaccines can cause a low-grade fever as the body builds immunity to the vaccine.\n\n7. Dehydration: Severe dehydration can cause a fever as the body struggles to regulate its temperature.\n\n8. Stress: Emotional or

In [7]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool

class LoggingCallbackHandler(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, invocation_params, **kwargs):
        print(f"Starting LLM with prompt: {prompts}")

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"Generated token: {token}")

# Define the symptom checker tool
@tool
def symptom_checker(symptom: str) -> str:
    """Check common causes for a given symptom."""
    common_causes = {
        "fever": "Common causes include infections (bacterial or viral), heat exhaustion, and inflammatory conditions.",
        "headache": "Common causes include tension, stress, dehydration, and sinusitis.",
        "cough": "Common causes include colds, flu, bronchitis, and asthma."
    }
    return common_causes.get(symptom.lower(), "Symptom not found in the database. Please consult a healthcare professional.")

# Instantiate the LLM with callbacks
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), streaming=True, callbacks=[LoggingCallbackHandler()])

# Define prompts
prompt1 = ChatPromptTemplate.from_template("Check common causes for the symptom: {symptom}")
prompt2 = ChatPromptTemplate.from_template("Summarize the advice in a friendly tone.")

# Create chains
chain1 = LLMChain(llm=llm, prompt=prompt1)
chain2 = LLMChain(llm=llm, prompt=prompt2)

# Invoking chains and printing results
response1 = chain1.invoke({"symptom": "headache"})
response2 = chain2.invoke({"symptom": response1})

print("Symptom Check:", response1)
print("Summary:", response2)



Starting LLM with prompt: ['Human: Check common causes for the symptom: headache']
Generated token: 
Generated token: Common
Generated token:  causes
Generated token:  for
Generated token:  the
Generated token:  symptom
Generated token:  of
Generated token:  headache
Generated token:  include
Generated token: :


Generated token: 1
Generated token: .
Generated token:  Stress
Generated token:  or
Generated token:  tension
Generated token: 

Generated token: 2
Generated token: .
Generated token:  De
Generated token: hydration
Generated token: 

Generated token: 3
Generated token: .
Generated token:  Lack
Generated token:  of
Generated token:  sleep
Generated token:  or
Generated token:  changes
Generated token:  in
Generated token:  sleep
Generated token:  patterns
Generated token: 

Generated token: 4
Generated token: .
Generated token:  Eye
Generated token:  strain
Generated token:  or
Generated token:  vision
Generated token:  problems
Generated token: 

Generated token: 5
Generated t

In [8]:
from langchain.evaluation import load_evaluator
from langchain.chat_models import ChatOpenAI

evaluator = load_evaluator("criteria", criteria="relevance", llm=ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY')))

eval_result = evaluator.evaluate_strings(prediction="Common causes include tension, stress, dehydration, and sinusitis.", input="Check common causes for the symptom: headache")
print(eval_result)


{'reasoning': 'Step by step reasoning:\n1. The input is asking to check common causes for the symptom "headache".\n2. The submission lists tension, stress, dehydration, and sinusitis as common causes for a headache.\n3. The submission is relevant to the input as it provides common causes for the symptom "headache".\n\nTherefore, the submission meets the criteria.', 'value': 'Y', 'score': 1}


In [9]:
custom_criteria = {
    "simplicity": "Does the language use brevity?",
    "bias": "Does the language stay free of human bias?",
    "clarity": "Is the writing easy to understand?",
    "truthfulness": "Is the writing honest and factual?"
}

evaluator = load_evaluator("criteria", criteria=custom_criteria, llm=ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY')))

eval_result = evaluator.evaluate_strings(input="Check common causes for the symptom: headache", prediction="Common causes include tension, stress, dehydration, and sinusitis.")
print(eval_result)


{'reasoning': '1. simplicity: \n- The language used in the submission is brief and to the point, listing out common causes for headaches in a concise manner.\n- The language does not include unnecessary details or complex wording.\n- Therefore, the submission meets the simplicity criterion.\n\n2. bias:\n- The submission lists common causes for headaches without introducing any personal opinions or prejudices.\n- The language does not show favoritism towards any particular cause.\n- Therefore, the submission stays free of human bias and meets the bias criterion.\n\n3. clarity:\n- The writing is easy to understand and straightforward.\n- Each cause for headaches is clearly listed.\n- The submission does not include confusing or unclear language.\n- Therefore, the writing is clear and meets the clarity criterion.\n\n4. truthfulness:\n- The submission lists common causes for headaches that are widely recognized in the medical field.\n- The causes mentioned are legitimate factors that can l